In [1]:
import os
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers.utils import load_image
from transformers import CLIPTextModel

c:\Users\igorg\Documents\Programação\PDSI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
INPUT_IMAGE_PATH = "C:\\Users\\igorg\\Documents\\Programação\\PDSI\\imagens\\kkk.jpeg"

LORA_FOLDER = "C:\\Users\\igorg\\Documents\\Programação\\PDSI\\output\\cyberpunk_style"

BASE_MODEL_ID = "stablediffusionapi/anything-v5"

In [11]:
TRIGGER_WORD = "cyberpunk_art"  # ou a palavra certa do seu LoRA

PROMPT = f"{TRIGGER_WORD}, cyborg sleeping, system recharge mode, wires connected to head, futuristic techwear armor, glowing blue neon lights, dark sci-fi lab background, low battery warning, high tech, cyberpunk 2077 vibes, masterpiece"

NEGATIVE_PROMPT = "daylight, sun, natural background, rustic, vintage, awake, alert eyes"

In [12]:
OUTPUT_FOLDER = "./resultados_style_CB"
SEED = 42
NUM_STEPS = 30
GUIDANCE_SCALE = 7.5
STRENGTH = 0.80

In [13]:
def run_style_transfer():
    print(f"--- Iniciando com GPU: {torch.cuda.get_device_name(0)} ---")
    
    # 1. Carregar Modelo Base
    print(f"Carregando modelo base...")
    
    # Clip Skip 2
    text_encoder = CLIPTextModel.from_pretrained(
        BASE_MODEL_ID, 
        subfolder="text_encoder", 
        num_hidden_layers=11
    )

    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        BASE_MODEL_ID,
        text_encoder=text_encoder,
        torch_dtype=torch.float16, 
        safety_checker=None
    ).to("cuda")

    # Otimizações de memória para a RTX 3050
    pipe.enable_attention_slicing()
    pipe.enable_model_cpu_offload() # <--- ESSA É A MÁGICA PARA 4GB/6GB VRAM

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    # 2. Carregar Imagem
    if not os.path.exists(INPUT_IMAGE_PATH):
        print(f"ERRO: A imagem '{INPUT_IMAGE_PATH}' não existe na pasta!")
        return

    init_image = load_image(INPUT_IMAGE_PATH).resize((512, 512))

    lora_files = sorted([f for f in os.listdir(LORA_FOLDER) if f.endswith(".safetensors")])
    print(f"Encontrados {len(lora_files)} checkpoints.")

    for lora_file in lora_files:
        lora_path = os.path.join(LORA_FOLDER, lora_file)
        print(f"--> Testando: {lora_file}")

        try:
            pipe.load_lora_weights(lora_path)
        except Exception as e:
            print(f"Erro ao carregar LoRA: {e}")
            continue

        generator = torch.Generator(device="cuda").manual_seed(SEED)
        
        # --- A CORREÇÃO ESTÁ AQUI: AUTOCAST ---
        # Isso converte automaticamente os tipos float32/float16 na hora certa
        with torch.autocast("cuda"):
            result = pipe(
                prompt=PROMPT,
                image=init_image,
                strength=STRENGTH,
                negative_prompt=NEGATIVE_PROMPT,
                num_inference_steps=28,
                guidance_scale=GUIDANCE_SCALE,
                generator=generator
            ).images[0]
        # --------------------------------------

        save_name = f"teste_{lora_file.replace('.safetensors', '')}.png"
        result.save(os.path.join(OUTPUT_FOLDER, save_name))
        print(f"Salvo: {save_name}")

        pipe.unload_lora_weights()

    print("\nSUCESSO! Verifique a pasta 'resultados_finais'.")

if __name__ == "__main__":
    run_style_transfer()

--- Iniciando com GPU: NVIDIA GeForce RTX 3050 Laptop GPU ---
Carregando modelo base...


Some weights of the model checkpoint at stablediffusionapi/anything-v5 were not used when initializing CLIPTextModel: ['text_model.encoder.layers.11.layer_norm1.bias', 'text_model.encoder.layers.11.layer_norm1.weight', 'text_model.encoder.layers.11.layer_norm2.bias', 'text_model.encoder.layers.11.layer_norm2.weight', 'text_model.encoder.layers.11.mlp.fc1.bias', 'text_model.encoder.layers.11.mlp.fc1.weight', 'text_model.encoder.layers.11.mlp.fc2.bias', 'text_model.encoder.layers.11.mlp.fc2.weight', 'text_model.encoder.layers.11.self_attn.k_proj.bias', 'text_model.encoder.layers.11.self_attn.k_proj.weight', 'text_model.encoder.layers.11.self_attn.out_proj.bias', 'text_model.encoder.layers.11.self_attn.out_proj.weight', 'text_model.encoder.layers.11.self_attn.q_proj.bias', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.11.self_attn.v_proj.bias', 'text_model.encoder.layers.11.self_attn.v_proj.weight']
- This IS expected if you are initializing CLIPTextMo

Encontrados 10 checkpoints.
--> Testando: cyberpunk_style-01.safetensors


Loading adapter weights from state_dict led to unexpected keys not found in the model: text_model.encoder.layers.11.mlp.fc1.lora_A.default_0.weight, text_model.encoder.layers.11.mlp.fc1.lora_B.default_0.weight, text_model.encoder.layers.11.mlp.fc2.lora_A.default_0.weight, text_model.encoder.layers.11.mlp.fc2.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.k_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.k_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.out_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.out_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.q_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.q_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.v_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.v_proj.lora_B.default_0.weight. 
100%|██████████| 22/22 [00:11<00:00,  2.00it/s]


Salvo: teste_cyberpunk_style-01.png
--> Testando: cyberpunk_style-02.safetensors


...

Salvo: teste_cyberpunk_style-08.png
--> Testando: cyberpunk_style-09.safetensors


Loading adapter weights from state_dict led to unexpected keys not found in the model: text_model.encoder.layers.11.mlp.fc1.lora_A.default_0.weight, text_model.encoder.layers.11.mlp.fc1.lora_B.default_0.weight, text_model.encoder.layers.11.mlp.fc2.lora_A.default_0.weight, text_model.encoder.layers.11.mlp.fc2.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.k_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.k_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.out_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.out_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.q_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.q_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.v_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.v_proj.lora_B.default_0.weight. 
100%|██████████| 22/22 [00:09<00:00,  2.21it/s]


Salvo: teste_cyberpunk_style-09.png
--> Testando: cyberpunk_style-10.safetensors


Loading adapter weights from state_dict led to unexpected keys not found in the model: text_model.encoder.layers.11.mlp.fc1.lora_A.default_0.weight, text_model.encoder.layers.11.mlp.fc1.lora_B.default_0.weight, text_model.encoder.layers.11.mlp.fc2.lora_A.default_0.weight, text_model.encoder.layers.11.mlp.fc2.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.k_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.k_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.out_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.out_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.q_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.q_proj.lora_B.default_0.weight, text_model.encoder.layers.11.self_attn.v_proj.lora_A.default_0.weight, text_model.encoder.layers.11.self_attn.v_proj.lora_B.default_0.weight. 
100%|██████████| 22/22 [00:09<00:00,  2.24it/s]


Salvo: teste_cyberpunk_style-10.png

SUCESSO! Verifique a pasta 'resultados_finais'.
